Questions :
* On doit toujours considérer la ruine?
* Possibilité de coder en C++ ?
* Pour la question 3, c'est quoi la loi de $\xi_n$?

**Tester le code avec des événements plus probable!!!**
## To do list:
* Q1-1
#####* Splitting (Diane) (faudrait jouer avec le nombre de niveaux de splitting, donc avec la valeur de la variable "seuil", pour voir ce qui donne la meilleure précision)
#####* Echantillonage d'importance (Peng-Wei)
#####* Chaîne de Markov -> Regarder l'article FodraPhamMicrostructure2015
#####* Intervalle de confiance pour la Monte-Carlo simple (et pour les autres ?)

* Q1-2
#####* Quantile processus AR(1) (comme dans calcul liste_a Q1-1) (Diane)
#####* Quantile échantillonnage d'importance (Diane)
* Q2
#####* La même chose mais il faut prendre en compte les différents instants de saut. Donc ça change tout.
#####* M-C (Peng-Wei

* Q
* Q3
# Finished :
* Q1
#####* Monte-Carlo naïve


In [0]:
import numpy as np
import math
import matplotlib.pyplot

## Q1

Dans cette première modélisation simplifiée, on considère donc que le prix $P_t$ est un processus de Poisson de paramètres $\lambda, \nu$ où $\nu$ est la loi des incréments $J_n$. 

Pour un temps d'attente moyen entre deux sauts de $300s$, on prend $\lambda = \dfrac{1}{300}$


In [0]:
# On identifie le processus par le processus de Poisson composé, qui finit à un temps fixé T
# On crée des fonctions pour modéliser le processus.

#On définit nu, la loi des incréments
saut_1 = lambda x: np.random.choice([-1, 1], size=x, replace=True, p=[0.5, 0.5]) #correspond à m=1
saut_2 = lambda x: np.random.choice([-3, -2, -1, 1, 2, 3], size=x, replace=True, p=0.5*np.array([1/6, 1/3, 1/2, 1/2, 1/3, 1/6])) #correspond à m=3
P0 = 35
T = 4*60*60
lamb = 1/300

### Q1 -1 Probabilité de ruine


In [0]:
#Méthode 4 - Simulation des temps entre les sauts plutôt que directement N puis uniforme

def liste_prix_sauts(lamb,T,P0,saut):
    t=np.random.exponential(lamb)
    liste_increments= []
    while t<T:
        liste_increments.append()
        t+= np.random.exponential(lamb)
    N = np.random.poisson(lamb*T) #le nombre de sauts suit une loi de Poisson de paramètre lambda*T
    liste_increments = saut(N)
    liste_prix = P0 + np.cumsum(liste_increments)

    return liste_prix


def MC_naif(M,lamb,T,P0,saut): 
    liste_bool_inf_0 = np.zeros(M) 

    for l in range(M):
        liste_prix = liste_prix_sauts(lamb,T,P0,saut) #prix juste après les sauts
        if np.min(liste_prix) <0:
            liste_bool_inf_0[l] = 1

    p_r = np.mean(liste_bool_inf_0)
    return p_r



In [0]:
## Monte-Carlo Naif

def liste_prix_sauts(lamb,T,P0,saut):
    N = np.random.poisson(lamb*T) #le nombre de sauts suit une loi de Poisson de paramètre lambda*T
    liste_increments = saut(N)
    liste_prix = P0 + np.cumsum(liste_increments)

    return liste_prix


def MC_naif(M,lamb,T,P0,saut): 
    liste_bool_inf_0 = np.zeros(M) 

    for l in range(M):
        liste_prix = liste_prix_sauts(lamb,T,P0,saut) #prix juste après les sauts
        if np.min(liste_prix) <0:
            liste_bool_inf_0[l] = 1

    p_r = np.mean(liste_bool_inf_0)
    return p_r



In [0]:
%%time

P0 = 35 #35 en théorie mais on commence par tester avec plus bas
T = 4*60*60 #conversion en secondes
lamb = 1/300
M = int(1e6)

p_r = MC_naif(M,lamb,T,P0,saut_2)

R_IC = 1.96*np.sqrt(p_r*(1-p_r))/np.sqrt(M) #rayon de l'intervalle de confiancee
print(p_r)
print(R_IC)

#donne 0.00254 pour P0=20 et M=10^5 et m=1(total time 5.02 s)
#m=3 P0=35 M=10^6: proba = 0.004341

0.004402
0.00012975470009396036
CPU times: user 58 s, sys: 11.9 ms, total: 58 s
Wall time: 58 s


In [0]:
## Monte-Carlo Naif avec numba

from numba import jit

@jit(nopython=True)
def inf_echantillon(N, J, res, P0): # Function is compiled to machine code when called the first time
  for i in range(len(N) - 1):
    somme = P0
  
    for j in range(N[i], N[i + 1]):
      somme += J[j]
      if somme < 0:
        res[i] = True
        break
  return res

def trajectoire(P0, T, lamb, saut, size):
  N = np.random.poisson(lam=lamb * T, size=size + 1)
  N[0] = 0
  N = N.cumsum()                    # La valeur N[i] - N[i - 1] est égale à Ni pour le i-ième échantillon
                                    # Donc la somme des sauts entre indice N[i] et N[i + 1] - 1 suit la loi voulue
  J = saut(N[-1] + 1)
  res = np.zeros(size, dtype=bool)
  res = inf_echantillon(N, J, res, P0)
  proba = np.mean(res)
  return proba

In [0]:
%%time

p_r = trajectoire(P0, T, lamb, saut_1, size=int(1e7))

R_IC = 1.96*np.sqrt(p_r*(1-p_r))/np.sqrt(M) #rayon de l'intervalle de confiance

print(p_r)
print(R_IC)

#print(trajectoire(P0, T, lamb, saut_1, size=int(1e7)))

#m=1
#0.00279 pour P0 = 20 M = 10^5 (total time 161 ms)
#5e-07 pour P0=35 et M=10^7 total time : 16.1 s
#quand on fait la moyenne sur 10 essais à M=10^7, on trouve proba_emp = 2.009975 e-7 et sigma_emp = 3.6 e-7 (tout ça pour m=1)

#m=3
#proba_emp=0.0043245, sigma_emp = 7.1273066441679e-05

5e-07
1.385928944643267e-06
CPU times: user 12.7 s, sys: 4.38 s, total: 17.1 s
Wall time: 17.1 s


In [0]:
## Splitting et MCMC - Méthode 3



def NiveauxSplitting(a,seuil,n,lamb,T,p,P0,saut):

    """
    Fonction qui renvoie une estimation des niveaux
    de splitting a_1, a_2, ..., a_k tels que P(Phi_T <= a_k | Phi_T <= a_{k-1}) = 0.1 = seuil
    (où Phi_T : inf de P_t pour t dans [0;T])
    Ces niveaux sont des quantiles d'une loi conditionnelle.
    On utilise l'inversion de la fonction de repartition empirique de 
    cette loi afin d'estimer un quantile par
    le quantile empirique.
    On a a = a_k < a_{k-1} < ... < a_0= + infini (dans notre problème, a = 0)
    La fonction renvoie quantiles = [a_1, ..., a_k]
    """
    ## Estimation du premier niveau a_1: c'est le 
    ## quantile d'une loi non conditionnelle.
    ## On l'estime ici par la methode ergodique

    liste_Phi = np.zeros(n)

    liste_sauts = liste_sts(lamb,T,saut)

    for l in range(n):
        coloriage = liste_sauts[:,np.random.binomial(1,p,size = len(liste_sauts[0])) ==1]

        liste_sauts_tilde = liste_sts((1-p)*lamb,T,saut)
        new_liste_sauts = np.concatenate((coloriage,liste_sauts_tilde),axis=1)
        new_liste_sauts = tri_temps(new_liste_sauts)

        liste_sauts = new_liste_sauts
        liste_Phi[l] = Phi(liste_sauts,P0)

    liste_Phi.sort()

    quantiles = np.array([liste_Phi[int(np.ceil(seuil*n))-1]])    

    while quantiles[-1] > a:
        print("Inside while")
        liste_Phi = np.zeros(n)

        
        while Phi(liste_sauts,P0)>=quantiles[-1]:
            liste_sauts = liste_sts(lamb,T,saut)
        ## Simulation du processus AR(1) conditionnel
    
        for l in range(n):
            coloriage = liste_sauts[:,np.random.binomial(1,p,size = len(liste_sauts[0])) ==1]

            liste_sauts_tilde = liste_sts((1-p)*lamb,T,saut)
            new_liste_sauts = np.concatenate((coloriage,liste_sauts_tilde),axis=1)
            new_liste_sauts = tri_temps(new_liste_sauts)

            if Phi(new_liste_sauts,P0)<quantiles[-1]:
                liste_sauts = new_liste_sauts

            liste_Phi[l] = Phi(liste_sauts,P0)
    
        liste_Phi.sort()
        quantiles = np.append(quantiles, liste_Phi[int(np.ceil(seuil*n))-1] )

    
    ## On selectionne les niveaux a_{k-1},..., a_1 strictement au dessus de a
    quantiles = quantiles[:-1]
    ## On rajoute a
    quantiles = np.append(quantiles,a)

    return quantiles


def Phi(liste_sauts,P0):#fonction qui renvoie l'inf des valeurs de X aux instants de saut
    #ie l'inf de P0+cumsum(incréments) 
    if len(liste_sauts[0]) == 0:
        return P0
    liste_prix = P0+np.cumsum(liste_sauts[1,:])
    prix_min = np.min(liste_prix)
    return prix_min

def liste_sts(lbda,T,saut):
    
    N = np.random.poisson(lbda*T)
    liste_temps_sauts = np.random.uniform(low = 0, high = T, size = N)
    liste_temps_sauts_triee = [np.sort(liste_temps_sauts)]
    liste_increments = [saut(N)]
    #renvoie un array de N colonnes et 2 lignes: 1ere ligne pour les temps des sauts (T_n), deuxième ligne pour leurs amplitudes (J_n)
    return np.concatenate((liste_temps_sauts_triee,liste_increments),axis=0)

def tri_temps(new_liste_sauts):
    ordre = [new_liste_sauts[0,:].argsort()]
    liste_sauts_triee = np.take_along_axis(new_liste_sauts, np.concatenate((ordre,ordre),axis=0), axis=1) 
    return liste_sauts_triee


def MCMC(M,p,lamb,liste_a,P0,saut): 

    liste_pi = np.zeros(len(liste_a)) #estimateurs des probabilités conditionnelles

    liste_indicatrices = np.zeros(M,dtype=bool) 
    """
    le k-ieme élém. de liste_indicatrices vaut True si le prix devient négatif avant l'instant T lors du k-ieme essai; False sinon
    """

    #Loi non conditionnelle
    
    liste_sauts = liste_sts(lamb,T,saut) 


    for l in range(M):
        coloriage = liste_sauts[:,np.random.binomial(1,p,size = len(liste_sauts[0])) ==1]

        liste_sauts_tilde = liste_sts((1-p)*lamb,T,saut)
        new_liste_sauts = np.concatenate((coloriage,liste_sauts_tilde),axis=1)
        new_liste_sauts = tri_temps(new_liste_sauts)

        liste_sauts = new_liste_sauts
        liste_indicatrices[l] = (Phi(liste_sauts,P0)<liste_a[0])

    liste_pi[0] = np.mean(liste_indicatrices)
    
        


    for k in range(1,len(liste_a)):
        liste_indicatrices = np.zeros(M,dtype=bool)  

        #Initialisation 

        while Phi(liste_sauts,P0)>=liste_a[k-1]:
          
            liste_sauts = liste_sts(lamb,T,saut)

      
        for l in range(M):
            coloriage = liste_sauts[:,np.random.binomial(1,p,size = len(liste_sauts[0])) ==1]

            liste_sauts_tilde = liste_sts((1-p)*lamb,T,saut)
            new_liste_sauts = np.concatenate((coloriage,liste_sauts_tilde),axis=1)
            new_liste_sauts = tri_temps(new_liste_sauts)

            if Phi(new_liste_sauts,P0)< liste_a[k-1]:
                liste_sauts = new_liste_sauts
            liste_indicatrices[l] = (Phi(liste_sauts,P0) < liste_a[k])


        liste_pi[k] = np.mean(liste_indicatrices)
    

    proba_prix_negatif = np.prod(liste_pi)
        
    return proba_prix_negatif

In [0]:
P0 = 35 
T = 4*60*60 #conversion en secondes
lamb =  1/300 
M = int(1e4) 

a=0


#Choix des paramètres pour les niveaux de splitting et la simulation par chaîne de Markov
seuil = 0.03
p=0.5

liste_a = NiveauxSplitting(a,seuil,M,lamb,T,p,P0,saut_1)
print(liste_a)
#Donne [22. 14.  8.  3.  0.] pour m=1 et seuil = 0.05
#Donne [20.  9.  1.  0.] pour m=3 et seuil = 0.2

Inside while
Inside while
Inside while
[20. 12.  5.  0.]


In [0]:
probas = np.zeros(10)

M = int(1e5)
n=10

for i in range(n):
    probas[i] = MCMC(M, p,lamb,liste_a,P0,saut_1)

proba_emp = np.mean(probas)
sigma_emp = np.std(probas)/np.sqrt(n) #écart-type empirique de la moyenne des n estimateurs
print(proba_emp)
print(sigma_emp)
#Réponse pour m=1 ie k=0 et P0=35 : proba = 3.333674930684664e-07, variance = 5 10^-8
#m=3 et P0=35: proba = 0.004309577501008395, variance = 0.00010259619121779947
#m=1 liste_a = [24. 17. 12.  7.  3.  0.] seuil =0.1 M=10^4 n=10 proba_emp=4.619226215866903e-07 sigma_emp= 9.561801048257746e-08
#m=1 liste_a= [20. 12.  5.  0.] seuil = 0.03 P0=35 M=10^4 n=10 proba_emp = 3.6218700696e-07 sigma_emp = 7.599237232859178e-08 
#same avec M=10^5 proba_emp = 3.4354401671979703e-07. sigma_emp= 1.3033966706232877e-08 LE MEILLEUR QU ON AIT

3.4354401671979703e-07
1.3033966706232877e-08


In [0]:
print("Estimation de la probabilité de ruine pour m=1 par méthode de Splitting/MCMC: {:09.8f}+/-{:09.8f}".format(proba_emp,sigma_emp))

Estimation de la probabilité de ruine pour m=1 par méthode de Splitting/MCMC: 0.00000034+/-0.00000001


###  Q1 - 2 Quantile


Pour les intervalles de confiance des quantiles, voir cours 2 p.3 (y a un TCL pour le quantile)

__Aussi, parle des approximations gaussiennes des poissons composés p6__

D'après le cours:

Si la loi des sauts est centrée et de variance $\sigma^2$
Si on note $N_T$ le nombre de sauts dans $[0,T]$,

$$N \sim \mathcal{P}(\lambda T)$$
Alors le prix à $T$, $X_T$,  converge en loi vers $\mathcal{N}(P_0,\lambda T \sigma^2)$ quand $\lambda T \rightarrow + \infty$

Ici on a $\lambda T = 48$

Ca serait intéressant de comparer les quantiles obtenus en Q1.2 avec les quantiles de la loi normale $\mathcal{N}(P_0,\lambda T \sigma^2)$, où $\sigma^2=1$ si $m=1$ et $\sigma^2 = ...$ si $m=3$ (normalement, devraient être proches)

In [0]:
#Monte-Carlo accéléré - Méthode 2

# Poisson composé naïf
def Poisson_compose(P0, T, lamb, saut, size):
  # saut : une fonction qui prend la taille de sortie comme entrée pour les sauts.
  N = np.random.poisson(lam=lamb * T, size=size + 1)
  N[0] = 0
  N = np.cumsum(N)                  # La valeur N[i] - N[i - 1] est égale à N pour le i-ième échantillon
                                    # Donc la somme des sauts entre indice N[i] + 1 et N[i + 1] suit la loi voulue
  J = saut(N[-1] + 1)               # Sauts
  J = np.cumsum(J)
  M = J[N[1:]] - J[N[:-1]]          # On veut la somme entre l'indice N[i] + 1 et N[i + 1]
  return P0 + M # échantillon de taille (size)

def quantile(echantillion, size, quant):
  index_bot = int(math.ceil(size * quant))
  index_top = int(math.ceil(size * (1 - quant)))
  echantillion.partition((index_bot, index_top))
  return echantillion[index_bot], echantillion[index_top]

def Question1_2_naive(size, quant, saut, P0, T, lamb):
  return quantile(Poisson_compose(P0, T, lamb, saut, size), size, quant)

In [0]:
%%time
size = int(1e6)
quant = 1e-4
print(Question1_2_naive(size, quant, saut_1, P0, T, lamb))

#Donne 9 et 61 pour P0 = 35 et m=1 et niveau = 10^-4 
#Donne 5 et 65 pour P0 = 35 et m=1 et niveau = 10^-5 
#Donne 3 et 68 pour P0 = 35 et m=1 et niveau = 10^-6
#Donne -13 et 83 pour P0 = 35 et m=2 et niveau = 10^-4 
#Donne -21 et 92 pour P0 = 35 et m=2 et niveau = 10^-5
#Donne -25 et 94 pour P0 = 35 et m=2 et niveau = 10^-6

(-6, 46)
CPU times: user 1.41 s, sys: 4 ms, total: 1.41 s
Wall time: 1.42 s


In [0]:
#MCMC (chaînes de Markov) - Méthode 3

def Quantile_ergodique(seuil,n,lamb,T,p,P0,saut): #renvoie le quantile de niveau "seuil" du processus de poisson composé

    liste_prix_final = np.zeros(n)

    liste_sauts = liste_sts(lamb,T,saut) 

    for l in range(n):
        coloriage = liste_sauts[:,np.random.binomial(1,p,size = len(liste_sauts[0])) ==1]

        liste_sauts_tilde = liste_sts((1-p)*lamb,T,saut)
        new_liste_sauts = np.concatenate((coloriage,liste_sauts_tilde),axis=1)
        liste_sauts = tri_temps(new_liste_sauts)

        liste_prix_final[l] = np.cumsum(liste_sauts[1,:])[-1] +P0

    liste_prix_final.sort()
  
    quantile = liste_prix_final[int(np.ceil(seuil*n))-1]    

  
    return quantile

In [0]:
4.310−3

def Quantile_naif(seuil,n,lamb,T,P0,saut):

    liste_P_t = np.zeros(n)
    for i in range(n):
        N = np.random.poisson(lamb*T)
        increments = saut(N)
        
        liste_P_t[i] = np.sum(increments)+P0
    liste_P_t = np.sort(liste_P_t)
    return liste_P_t[int(np.ceil(n*seuil))-1]

In [0]:
P0 = 35
T = 4*60*60 #conversion en secondes
lamb =  1/300 
n = int(1e6) 

niveau = 1e-5
p=0.5

quantileGauche = Quantile_naif(niveau,n,lamb,T,P0,saut_2) #mettre Quantile_ergodique(niveau,n,lamb,T,p,P0,saut) ou Quantile_naif(niveau,n,lamb,T,P0,saut)
quantileDroit = Quantile_naif(1-niveau,n,lamb,T,P0,saut_2)
print("Quantile de niveau {} : {}".format(niveau,quantileGauche))
print("Quantile de niveau {} : {}".format(1-niveau,quantileDroit))

#Résultats pour Quantile_ergodique:
#Donne 9 et 61 pour P0 = 35 et k=0 et niveau = 10^-4 (c'est à peu près sym/P0 ce qui est rassurant)
#Donne 5 et 66 pour P0 = 35 et k=0 et niveau = 10^-5
#Donne 5 et 63 pour P0 = 35 et k=0 et niveau = 10^-6 (problème... faut augmenter n?) n : 10^5 -> 10^6 : on trouve 2 et 68 ; OK :D

#-15 et 83 pour P0=35 et k=1 et niveau = 10^-4 (sauts plus grands dc on va plus loin, logique; sym/0, logique)
#-16 et 90 pour P0=35 et k=1 et niveau = 10^-5
#Donne -25 et 97 pour P0 = 35 et k=1 et niveau = 10^-6 (en augmentant n de 10^5 à 10^6)

#Résultats Quantile_naïf (plus lent) (n=10^5):
#9 et 62
#5 et 66
#3 et 67
#-13 et 84
#-22 et 87 si on garde n=10^5, -20 et 89 si on passe à n=10^6
# -22 et 97(on passe à n=10^6)

Quantile de niveau 1e-05 : -20.0
Quantile de niveau 0.99999 : 89.0


In [0]:
#Calcul quantile par échantillonnage d'importance
#Voir slides amphi 2 p.12 pour calcul quantile avec changement de probabilité
#Ensuite voir amphi 3 pour changements de proba Poisson composés
#Différence importance sampling/échantillonage d'importance et changement de probabilité ?
#Transfo d'Esscher

##  Q2




### Q2-1 Probabilité de ruine

In [0]:
#MC Naif - Méthode 1

def ruine(lamb1,lamb2,T,P0,saut):
    N1 = np.random.poisson(lamb1*T)
    N2 = np.random.poisson(lamb2*T)
    ind_2 = np.sort(np.random.choice(np.arange(N1+N2),replace=False,size=N2)) #indices des sauts du processus 2
    one_first = np.random.binomial(1,0.5) #pour savoir si le premier incrément du processus 2 vaut 1 ou -1

    increments=saut(N1+N2)

    for i in range(N2):
        increments[ind_2[i]] = -1+ 2*((i+one_first)%2)
    liste_prix = P0 + np.cumsum(increments)

    inf_P = np.min(liste_prix)
    if inf_P<0:
        return 1
    return 0


def MC_naif(M,lamb1,lamb2,T,P0,saut): 
    liste_ruine = np.zeros(M) 

    for l in range(M):
        liste_ruine[l] = ruine(lamb1,lamb2,T,P0,saut) 


    proba_prix_negatif = np.mean(liste_ruine)
    return proba_prix_negatif


In [0]:
%%time
P0=35
lamb1 = 1/660
lamb2 = 1/110
T = 4*3600
M=int(1e6)

p_r = MC_naif(M,lamb1,lamb2,T,P0,saut_2)
R_IC = 1.96*np.sqrt(p_r*(1-p_r)/M)
print(p_r)
print(R_IC)

#pour m=3 M=10^5 P0=35: (7e-05, 5.1854910679703226e-05) = (p_r, ecart type empirique). 
#C'est bcp plus faible qu'avant (10^-3). Logique vu que tendance à revenir vers l'équilibre
#pour P0=5 ça donne bien la même chose que MC Naif Numba de PWC
#m=3 P0=35 M=10^6 : p_r = 5.9e-05, R_IC=1.505460153542431e-05

5.9e-05
1.505460153542431e-05
CPU times: user 2min 33s, sys: 25.4 ms, total: 2min 33s
Wall time: 2min 33s


In [0]:
#Monte-Carlo accéléré - Méthode 2

# On identifie le processus par le processus de Poisson composé, qui finit à un temps fixé T
# On crée des fonctions pour modéliser le processus.

saut_1 = lambda x: np.random.choice([-1, 1], size=x, p=[0.5, 0.5])
saut_2 = lambda x: np.random.choice([-3, -2, -1, 1, 2, 3], size=x, p=0.5*np.array([1/6, 1/3, 1/2, 1/2, 1/3, 1/6]))
P0 = 35
T = 4*60*60
lamb1 = 1/660
lamb2 = 1/110

from numba import jit

@jit(forceobj=True)
def Q2_1_MC(P0, T, lamb1, lamb2, saut):
  # saut : une fonction qui prend la taille de sortie comme entrée pour les sauts.
  N1 = np.random.poisson(lam=lamb1 * T)
  T1 = np.sort(np.random.uniform(low=0.0, high=T, size=N1))
  J1 = saut(N1)

  N2 = np.random.poisson(lam=lamb2 * T)
  T2 = np.sort(np.random.uniform(low=0.0, high=T, size=N2))

  J2_0 = saut_1(1)[0]

  somme = P0
  index_T2 = 0
  for i in range(len(T1)):
    while index_T2 < N2 and T2[index_T2] < T1[i]:
      somme += J2_0
      if somme < 0:
        return 1
      J2_0 *= -1
      index_T2 += 1
    somme += J1[i]
    if somme < 0:
      return 1
  return 0                      # 1 échantillon

def Q2_1_MC_n(P0, T, lamb1, lamb2, saut, size):
  somme = 0
  for i in range(size):
    somme += Q2_1_MC(P0, T, lamb1, lamb2, saut)
  return somme / size

In [0]:
%%time
P0 = 35
M=int(1e6)

p_r = Q2_1_MC_n(P0, T, lamb1, lamb2, saut_2, M)

R_IC = 1.96*np.sqrt(p_r*(1-p_r))/np.sqrt(M) #rayon de l'intervalle de confiance

print("Estimation de la probabilité de ruine pour m=3 par méthode de Monte-Carlo accélérée: {:09.8f}+/-{:09.8f}".format(p_r,R_IC))

#P0=35, M=10^5, m=3: p_r = 3 10^-5 et R_IC = 3.4 10^-5 => faut augmenter M
#Estimation de la probabilité de ruine pour m=3 par méthode de Monte-Carlo accélérée: 0.00005200+/-0.00001413

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 21 µs
Estimation de la probabilité de ruine pour m=3 par méthode de Monte-Carlo accélérée: 0.00005200+/-0.00001413


### Q2-2 Quantile

In [0]:
#Naif - Méthode 1


def Quantile_naif_2(seuil,n,lamb1,lamb2,T,P0,saut):

    liste_P_t = np.zeros(n)
    for i in range(n):
        N1 = np.random.poisson(lamb1*T)
        N2 = np.random.poisson(lamb2*T)
        P2 = 0 
        if (N2%2!=0): #si N2 est pair, la contribution du processus 2 vaut 0
            P2=-1+2*np.random.binomial(1,0.5)
        increments = saut(N1)
        
        liste_P_t[i] = np.sum(increments)+P0 +P2
    liste_P_t = np.sort(liste_P_t)
    return liste_P_t[int(np.ceil(n*seuil))-1]

In [0]:
#MC Accéléré - Méthode 2 (pas plus rapide ?)

# On identifie le processus par le processus de Poisson composé, qui finit à un temps fixé T
# On crée des fonctions pour modéliser le processus.


def Q2_2_MC(P0, T, lamb1, lamb2, saut):
  # saut : une fonction qui prend la taille de sortie comme entrée pour les sauts.
  N1 = np.random.poisson(lam=lamb1 * T)
  T1 = np.sort(np.random.uniform(low=0.0, high=T, size=N1))
  J1 = saut(N1)

  N2 = np.random.poisson(lam=lamb2 * T)
  T2 = np.sort(np.random.uniform(low=0.0, high=T, size=N2))

  J2_0 = saut_1(1)[0]

  somme = P0
  index_T2 = 0
  for i in range(len(T1)):
    while index_T2 < N2 and T2[index_T2] < T1[i]:
      somme += J2_0
      J2_0 *= -1
      index_T2 += 1
    somme += J1[i]
  return somme                      # 1 échantillon

def Q2_2_MC_n(P0, T, lamb1, lamb2, saut, size):
  res = []
  for i in range(size):
    res.append(Q2_2_MC(P0, T, lamb1, lamb2, saut))
  return np.array(res)

def quantile(echantillion, size, quant):
  index_bot = int(math.ceil(size * quant))
  index_top = int(math.ceil(size * (1 - quant)))
  echantillion.partition((index_bot, index_top))
  return echantillion[index_bot], echantillion[index_top]

def Question2_2_naive(size, quant, saut, P0, T, lamb1, lamb2):
  return quantile(Q2_2_MC_n(P0, T, lamb1, lamb2, saut, size), size, quant)

In [0]:
%%time 

P0 = 35

seuil = 1e-6
n = int(1e6)
print(Question2_2_naive(n, seuil, saut_2, P0, T, lamb1, lamb2))

#(19, 52) pour m=1 seuil 10^-4 n =10^4
#(14,56) pour m=1 seuil 10^-5 n =10^5
#Donne ... pour n = 10^6 et seuil=1e-6 et m=1
#Donne (0, 69) pour n = 10^4 et seuil=1e-4 et m=3
#Donne (-2, 71) pour n = 10^5 et seuil=1e-5 et m=3
#Donne pour n = 10^5 et seuil=1e-6 et m=3

(-5, 81)
CPU times: user 3min 51s, sys: 203 ms, total: 3min 52s
Wall time: 3min 52s


In [0]:
%%time

P0 = 35

seuil = 1e-6
n=int(1e6)

quantiles = Quantile_naif_2(seuil,n,lamb1,lamb2,T,P0,saut_1),Quantile_naif_2(1-seuil,n,lamb1,lamb2,T,P0,saut_1)
print(quantiles)
#Donne 17/55 pour n = 10^4 et seuil=1e-4 et m=1
#Donne (12.0, 56.0) pour n = 10^5 et seuil=1e-5 et m=1
#Donne (11.0, 58.0) pour n = 10^6 et seuil=1e-6 et m=1
#Donne (2.0, 65.0) pour n = 10^4 et seuil=1e-4 et m=3
#Donne (-4.0, 73.0) pour n = 10^5 et seuil=1e-5 et m=3
#Donne (-10.0, 77.0) pour n = 10^6 et seuil=1e-6 et m=3

(11.0, 58.0)
CPU times: user 1min 35s, sys: 16.9 ms, total: 1min 35s
Wall time: 1min 35s


## Q

Afin de modéliser le phénomène de retour vers la moyenne, on veut que le signe de $\hat{J_n}$ soit différent que celui de $\hat{J_{n - 1}}$. Mathématiquement, ça s'écrit :
$$\mathbb{P}(\hat{J_{n - 1}} = 1, \hat{J_{n}} = 1) < \mathbb{P}(\hat{J_{n - 1}} = 1, \hat{J_{n}} = -1)$$
et 
$$\mathbb{P}(\hat{J_{n - 1}} = -1, \hat{J_{n}} = -1) < \mathbb{P}(\hat{J_{n - 1}} = -1, \hat{J_{n}} = 1)$$
En identifiant avec la matrice de transition $\hat{Q}$, on obtient :
$$\frac{1 - \alpha_+}{2} > \frac{1 + \alpha_+}{2}, \frac{1 - \alpha_-}{2} > \frac{1 + \alpha_-}{2}$$
Ainsi, $\alpha_+ < 0$ et $\alpha_- < 0$.

## Q3

### Estimation de la probabilité d'un prix négatif

On implémentera deux méthodes pour cette question :

* Monte-Carlo naïf
* méthodes des particules en interaction

__Méthode des particules en interaction appliquée au problème__

Avec cette nouvelle modélisation des sauts et pour un nombre de sauts N fixé, on peut simuler les trajectoires du prix comme des chaînes de Markov, grâce à la méthode des particules en interaction. 

En effet, on cherche à calculer $\mathbb{E}[f(P_0, \cdots, P_N)]$, où $f(p_0,\cdots, p_N) = \mathbb{1}_{min(p_0,\cdots,x_N)<0}$

La suite des couples (prix, signe du saut) $(P_n,\hat{J_n})_{n \in \mathbb{N}}$ est définie par :

* $\hat{J_1}\sim -1 + 2 \mathcal{B}(1/2)$ 
* $P_1 = 35+ \hat{J_1}$
* $\forall n \geq 1, \hat{J}_{n+1} = \hat{J_n} \times \delta_n$ , où les $\delta_n$ sont iid suivant $-1+2 \mathcal{B} \Big(\dfrac{1+\alpha}{2}\Big) $ (valent 1 avec probabilité $\dfrac{1+\alpha}{2} $, -1 sinon)
* $X_{n+1} = X_{n} + \xi_n \times \hat{J}_{n+1} $

Les amplitudes de saut $\xi_n$ sont bien indépendantes identiquement distribuées, et indépendante de $(P_0, \hat{J}_0)$. On a bien une récurrence aléatoire, donc $(P_n,\hat{J_n})_{n \in \mathbb{N}}$ est une chaîne de Markov.

La probabilité d'avoir un prix négatif dans les $N$ premiers sauts est donnée par : 

$p_N= \mathbb{E}\Big[f(\,(P_0,\hat{J}_0), \cdots, (P_N,\hat{J}_N)\,)\Big]$ où $f((p_0,\hat{j_0},\cdots, (p_N,\hat{j_N})) = \mathbb{1}_{min(p_0,\cdots,x_N)<0}$

On simule donc une trajectoire de la chaîne de Markov $(P_n,\hat{J_n})_{n \in \mathbb{N}}$ jusqu'à n = N.

On choisit comme fonction de poids $G$ une fonction qui favorise les trajectoires où le prix $P_n$ est bas : $G_k(P_{0:k}) = exp(-\mu P_k)$. Il va falloir ajuster le paramètre $\mu \in \mathbb{R}^+$.

Une autre possibilité aurait été de choisir une fonction $G$ qui favorise les trajectoires croissantes, de la forme $G_k(P_{0:k}) = exp(-\mu (P_k-P_{k-1})$. Dans notre cas c'est une très mauvaise idée, car une trajectoire qui vient de descendre aura tendance à remonter ensuite. On sélectionnerait donc les trajectoires qui ont le plus de chance de monter au prochain saut, ce qui ne nous aiderait pas du tout à atteindre un prix négatif.

In [0]:
P0 = 35
T = 4*60*60
lamb = 1/300
N=100
alpha = -0.875

saut_1 = lambda x: np.ones(shape=x) #correspond à m=1
saut_2 = lambda x: np.random.choice([1, 2, 3], size=x, replace=True, p=np.array([1/2, 1/3, 1/6])) #correspond à m=3
saut_1_vect = lambda x,y : np.ones(shape=(x,y))
saut_2_vect = lambda x,y: np.random.choice([1, 2, 3], size=(x,y), replace=True, p=np.array([1/2, 1/3, 1/6])) 

In [0]:
def part_interaction(N,saut,P0,M,alpha,mu):
    #mu: paramètre des poids G_p
    # Tirages de M x N v.a. suivant la loi "saut" et de MxN v.a. delta_n représentant le changement de signe du saut par rapport au signe du saut précédent
    # seront utilisees pour construire les chaînes de Markov
    xi= saut(M, N)   #N: horizon temporel (nombre de sauts)
    delta = -1+2*np.random.binomial(1,(1+alpha)/2,size=(M,N-1))

    ###############################################################################
    ## En ponderant les trajectoires hautes : Estimation via G(X)=e^{alpha X_p}
    ###############################################################################

    # P stocke TOUTES LES VALEURS pour chacune des M chaines, dans l'optique du calcul de G_p 
    # J_hat stocke la dernière valeur du signe de saut (saut vers le bas ou vers le haut) pour chacune des M chaînes
    P = np.zeros((M,N))

    J_hat = -1+2*np.random.binomial(1,0.5,size=M) #le premier saut est vers le bas ou vers le haut avec la même probabilité

    # On initialise avec la valeur du prix après le premier saut
    P[:,0] = P0+J_hat*xi[:,0]
        
    facteurDroite = 1 #produit des moyennes des pondérations à chaque itération

    for k in range(N-1):
                
        weights = np.exp(-mu*P[:,k])
        
        facteurDroite *= np.mean(weights)
        
        weights = weights/np.sum(weights)

        I = np.random.choice(np.arange(M),size=M,replace=True,p=weights)
        
        #sélection
        P = P[I,:]
        J_hat=J_hat[I]

        #mutation

        J_hat=J_hat*delta[:,k]
        P[:,k+1] = P[:,k] +xi[:,k+1]*J_hat

    print("prix final=",P[:10,-1]) #pour ajuster mu; il faut que le prix final soit souvent négatif mais pas qu'il parte trop loin dans les négatifs
    tabFacteursGauche = np.exp(mu*np.sum(P[:,:-1],axis=1)) #tableau des 1/(\prod_{k=0}^{N-2} G_k)
    proba_negatif = np.sum(tabFacteursGauche[np.min(P,axis=1)<0])/M * facteurDroite
    return proba_negatif

In [0]:
#Monte-Carlo naïf - Méthode 1

def Q3_proba_negatif(N,saut,P0,M,alpha): 
    liste_px_inf_0 = np.zeros(M)
    for i in range(M):
        increments = saut(N)
        signes = -1+2*np.random.binomial(1,(1+alpha)/2,size=N)

        signes[0] = -1+np.random.binomial(1,0.5)*2

        signes = np.cumprod(signes)
        prix = P0 +np.cumsum(increments*signes)

        if np.min(prix)<0:
            liste_px_inf_0[i] = 1
        else:
            liste_px_inf_0[i] = 0
    return np.mean(liste_px_inf_0)


In [0]:
M= int(1e5)
P0= 35
N=100
mu = 0.4/P0 #ajusté empiriquement

#proba_negatif = Q3_proba_negatif(N,saut_2,P0,M,alpha)
proba_negatif = part_interaction(N,saut_2_vect,P0,M,alpha,mu)
print(proba_negatif)
#Pour P0=10 et m=3 et M=10^5 et N=100, donne 0.2262 avec MC naif
#Donne 0.2331 avec méthode particules en interaction (mu=0.03/P0)
#pour P0=35, particules en interaction donne 5.027933935719282e-05

prix final= [-10.  15.  -3.  -3.   6.  -4.  -2.  -7.   0.  -5.]
2.8925384894140467e-05


### Estimation de quantiles du prix final après N sauts

In [0]:
#Quantile simple 


def Quantile_naif_2(seuil,N,saut,P0,M,alpha):

    liste_P_t = np.zeros(M)
    for i in range(M):
        signes = -1+2*np.random.binomial(1,(1+alpha)/2,size=N)
        signes[0] = -1+np.random.binomial(1,0.5)*2
        signes = np.cumprod(signes)
        increments = saut(N)
        
        liste_P_t[i] = np.sum(increments*signes)+P0 
    liste_P_t = np.sort(liste_P_t)
    return liste_P_t[int(np.ceil(M*seuil))-1],liste_P_t[int(np.ceil(M*(1-seuil)))-1]

In [0]:
seuil = 1e-4
M=int(1e4)
nbIter = 10
quantiles = np.zeros((nbIter,2))

for i in range(nbIter):
    quantiles[i][0],quantiles[i][1] = Quantile_naif_2(seuil,N,saut_1,P0,M,alpha)

estimateur_quantiles = np.mean(quantiles,axis=0)
erreur = np.std(quantiles,axis=0)
print("Quantiles de niveau {} : ({} +/ {:08.7f}, {} +/ {})".format(alpha,estimateur_quantiles[0],erreur[0],estimateur_quantiles[1],erreur[1]))

Quantiles de niveau -0.875 : (24.2 +/ 0.9797958971132713, 45.4 +/ 1.2)


In [0]:
print("Quantiles de niveau {} : ({} +/ {:04.3f}, {} +/ {})".format(alpha,estimateur_quantiles[0],erreur[0],estimateur_quantiles[1],erreur[1]))

Quantiles de niveau -0.875 : (24.2 +/ 0.980, 45.4 +/ 1.2)


### Limite macroscopique

In [0]:
#Monte-Carlo 

def Q3_proba_negatif(lamb,T,saut,P0,M,alpha): 
    liste_px_inf_0 = np.zeros(M)
    for i in range(M):
        N = np.random.poisson(lamb*T)
        increments = saut(N)
        signes = -1+2*np.random.binomial(1,(1+alpha)/2,size=N)
        signes[0] = -1+np.random.binomial(1,0.5)*2
        signes = np.cumprod(signes)
        prix = P0 +np.cumsum(increments*signes)
        if np.min(prix)<0:
            liste_px_inf_0[i] = 1
        liste_px_inf_0[i] = 0
    return np.mean(liste_px_inf_0)

In [0]:
T = 8*250*24*60*60
lamb = 1/300
M = int(1e3)
P0=1000

proba_negatif = Q3_proba_negatif(lamb,T,saut_2,P0,M,alpha)
print(proba_negatif)

0.0
